Make this notebook as wide as the browser window allows:

In [82]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Functions to read the MessagePack binary file:

In [71]:
import msgpack
import numpy as np

def ext_hook(code, data):
    """Read code-2 data as float32 numpy arrays."""
    if code == 2:
        return np.frombuffer(data, dtype=np.float32)
    return ExtType(code, data)


def load_summary(path):
    """Load summary data from a file."""
    with open(path, "rb") as fp:
        raw_bytes = fp.read()

    return msgpack.unpackb(
        raw_bytes, strict_map_key=False, use_list=False, ext_hook=ext_hook
    )

Helper functions for querying summary data:

In [77]:
import datetime

def get_dates(summary):
    """Extract a list of dates from a summary."""
    time_data = summary["time"]
    if "YEAR" in time_data and "MONTH" in time_data and "DAY" in time_data:
        # easy path first
        return [
            datetime.datetime(y, m, d)
            for (y, m, d) in zip(
                time_data["YEAR"]["values"],
                time_data["MONTH"]["values"],
                time_data["DAY"]["values"],
            )
        ]
    else:
        # do all the work ourselves
        start_date = datetime.datetime(*reversed(summary["start_date"]))
        return [
            start_date + datetime.timedelta(seconds=int(d * 86400))
            for d in time_data["TIME"]["values"]
        ]
    
def common_keys(summaries):
    """Recursively extract common keys from a list of summaries."""
    if any(not isinstance(s, dict) for s in summaries) or "unit" in summaries[0]:
        return None

    keys = set(summaries[0].keys())
    for s in summaries[1:]:
        keys &= s.keys()

    return {k: common_keys([d[k] for s in summaries]) for k in keys}

Getting ready to plot:

In [89]:
import matplotlib.pyplot as plt

def plot_summary_keyword(summary, keys):
    """Plots data from a summary given a path of keys to extract it."""
    dates = get_dates(summary)
    
    for key in keys:
        summary = summary[key]
    values = summary['values']
    
    f, ax = plt.subplots(1, 1, figsize = (15, 10))
    ax.plot(dates, values, 'o-', markevery=0.02);
    ax.grid()
    ax.set_xlabel(f'Date');
    ax.set_ylabel(f'{keys[-1]} [{summary["unit"]}]');

    # adjust all font sizes in one sweeping motion
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)

This class will store all our summary data and selection widgets

In [118]:
import ipywidgets
from traitlets import HasTraits, Dict, Instance

class DataManager(HasTraits):
    loaded_summary_data = Dict()
    
    type_selector = Instance(ipywidgets.ToggleButtons)
    loc_selector = Instance(ipywidgets.Select)
    kw_selector = Instance(ipywidgets.Select)

    def add_summary(self, path):
        self.loaded_summary_data[path] = load_summary(path)
    
    def common_keys(self, names):
        return common_keys([self.loaded_summary_data[n] for n in names])
    
    def get_dates(self, name):
        summary = self.loaded_summary_data[name]
        return get_dates(summary)
    
    def unload_files(self, paths):
        for p in paths:
            del self.loaded_summary_data[p]

dm = DataManager()

In [94]:
from ipyfilechooser import FileChooser

fc = FileChooser('.', use_dir_icons=True)
fc.title = '<b>Select an .mpk file to open</b>'    
fc.register_callback(lambda w: dm.add_summary(w.selected))

display(fc)

FileChooser(path='.', filename='', show_hidden='False')

In [107]:
tb = widgets.ToggleButtons(
    options=[],
    description='Type:',
    disabled=False,
    button_style='',
)

In [108]:
display(tb)

ToggleButtons(description='Type:', options=(), value=None)

In [110]:
tb.options=['kaka', 'puka']